In [2]:
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
from tqdm.asyncio import tqdm
import os, re, asyncio
import time
import nest_asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:

RAW_PATH    = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/sampled_data_full_fields_clean.jsonl"
LABELED_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/LLM_labels.jsonl"
REVIEW_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/to_review.jsonl"

N_TO_LABEL = 1000          # nbr offres à labelliser
MAX_WORKERS = 5          # threads parallèles (limite API)
MAX_CHARS = 10000         # limite tokens, strat utile

# ======== CLEAN TEXT ========
def clean_and_truncate(x):
    if isinstance(x, str):
        x = (x.replace("<p>", " ").replace("</p>", " ")
               .replace("<br/>", " ").replace("\n", " ").strip())
        return x[:MAX_CHARS]
    return ""

# ======== PROMPTS ========
SYSTEM_PROMPT = """
You are an expert HR analyst specialized in job offer analysis.

Your ONLY task is to output STRICT JSON with these EXACT keys:
DIVERSITY, REMUNERATION_BENEFITS, PROFESSIONAL_OPPORTUNITIES,
CULTURE_VALUES, LEADERSHIP, WORK_LIFE_BALANCE, CONFIDENCE.

You MUST NOT write any explanation, comments, or text outside the JSON.

Here is how to assign each label (ONLY ONE 1 or 0 each):

"DIVERSITY":
Repérez si l'offre mentionne l'une des formes de diversité :
- Handicap (accessibilité, adaptations raisonnables, etc.)
- Inclusion & équité (égalité des chances, non-discrimination, etc.)
- Genre (égalité des genres, LGBTQ+, etc.)
- Âge (diversité générationnelle)
Répondez 1 si au moins un de ces points est présent, sinon 0. AUCUN autre texte.

"REMUNERATION_BENEFITS":
Cherchez :
- Salaire, primes, bonus
- Mutuelle, avantages sociaux, retraite
Répondez 1 si au moins un de ces points est présent, sinon 0.

"PROFESSIONAL_OPPORTUNITIES":
Cherchez :
- Formation, mentorat, développement des compétences
- Évolution de carrière, nouveaux projets, mobilité
Répondez 1 si au moins un de ces points est présent, sinon 0.

"CULTURE_VALUES":
Repérez :
- Culture entreprise, valeurs, collaboration, mission, esprit d’équipe
Répondez 1 si au moins un de ces points est présent, sinon 0.

"LEADERSHIP":
Cherchez :
- Mentions de dirigeants, vision stratégique, style de management
Répondez 1 si au moins un de ces points est présent, sinon 0.

"WORK_LIFE_BALANCE":
Repérez :
- Télétravail, horaires flexibles
- Bien-être / équilibre vie pro-vie perso / congés parentaux
Répondez 1 si au moins un de ces points est présent, sinon 0.

You MUST RETURN ONLY valid JSON like:
{
  "DIVERSITY": 0 or 1,
  "REMUNERATION_BENEFITS": 0 or 1,
  "PROFESSIONAL_OPPORTUNITIES": 0 or 1,
  "CULTURE_VALUES": 0 or 1,
  "LEADERSHIP": 0 or 1,
  "WORK_LIFE_BALANCE": 0 or 1,
  "CONFIDENCE": integer 0-100
}
If unsure → still output JSON with 0 everywhere.
ANY OTHER OUTPUT IS WRONG.
"""

def build_user_prompt(row):
    return f"""
Title: {row.get('job_title')}
Company: {row.get('entreprise_nom')}
Location: {row.get('location_label')}
Contract: {row.get('contractType')}
Salary: {row.get('salary_min')} - {row.get('salary_max')} ({row.get('salary_period')})

Job Description:
{clean_and_truncate(row.get('description_job'))}

Profile Required:
{clean_and_truncate(row.get('description_profil'))}

Full Description:
{clean_and_truncate(row.get('description_full'))}

Return ONLY JSON with 7 keys.
"""

# ======== JSON EXTRACTION ROBUSTE (IMPORTANT) ========
def extract_json(text):
    """Isoler le JSON même si GPT écrit autour."""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None

# ======== API CALL ========

def call_llm(row):
    prompt = build_user_prompt(row)
    t0 = time.time()

    try:
        response = client.chat.completions.create(
            model="gpt-5-nano",
            messages=[{"role": "system", "content": SYSTEM_PROMPT},
                      {"role": "user", "content": prompt}],
        )
        content = response.choices[0].message.content
        try:
            j = json.loads(content)
            return row, j, time.time() - t0
        except:
            return row, None, time.time() - t0

    except Exception as e:
        return row, None, time.time() - t0


# ========= LOAD RAW DATA =========
df_raw = pd.read_json(RAW_PATH, lines=True)
print(f"📦 Total dans RAW : {len(df_raw)}")

# ========= AVOID RE-LABELING =========
if os.path.exists(LABELED_PATH):
    df_labeled = pd.read_json(LABELED_PATH, lines=True)
    done_urls = set(df_labeled["url"].unique())
else:
    done_urls = set()

# ========= SELECT ONLY NEW ONES =========
df_todo = df_raw[~df_raw["url"].isin(done_urls)].head(N_TO_LABEL).copy()
print(f"🚀 A traiter : {len(df_todo)}")

if len(df_todo) == 0:
    print("🎉 Rien à faire - tout est déjà labelisé!")
    exit()

# ========= APPEND FILES =========
f_out = open(LABELED_PATH, 'a', encoding='utf-8')
f_rev = open(REVIEW_PATH, 'a', encoding='utf-8')


# ========= RUN =========
rows = df_todo.to_dict(orient="records")
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [executor.submit(call_llm, row) for row in rows]
    for f in tqdm(as_completed(futures), total=len(rows)):
        row, res, t_api = f.result()
        print(f"⚙️ {t_api:.2f}s  | {row.get('url')}")

        if res is not None and res.get("CONFIDENCE", 0) >= 50:
            row["Label_All"] = res
            json.dump(row, f_out, ensure_ascii=False)
            f_out.write("\n")
            f_out.flush()
        else:
            row["LLM_PROPOSED_LABELS"] = res
            json.dump(row, f_rev, ensure_ascii=False)
            f_rev.write("\n")
            f_rev.flush()

f_out.close()
f_rev.close()

print("\n🎯 FIN DU RUN – aucune donnée écrasée.")
print("➡ Tu peux relancer 100x, il reprendra automatiquement où tu t'es arrêté.")

📦 Total dans RAW : 50000
🚀 A traiter : 1000


  0%|          | 1/1000 [00:08<2:16:07,  8.18s/it]

⚙️ 8.22s  | https://www.cadremploi.fr/emploi/detail_offre?offreId=1567662492989386


  0%|          | 2/1000 [00:09<1:09:09,  4.16s/it]

⚙️ 9.57s  | https://www.cadremploi.fr/emploi/detail_offre?offreId=1567637897958480


...

⚙️ 9.49s  | https://www.jobintree.com/offre-emploi/all-the-top-bananas-7936/leader-atelier-thermoformage-20671618


100%|█████████▉| 999/1000 [46:37<00:02,  2.29s/it]

⚙️ 14.68s  | https://www.jobintree.com/offre-emploi/pole-emploi-7915/technicien-technicienne-controle-et-mesure-en-mecanique-20306585


100%|██████████| 1000/1000 [46:39<00:00,  2.80s/it]

⚙️ 10.88s  | https://www.jobintree.com/offre-emploi/all-the-top-bananas-7936/biologiste-manager-groupe-prive-lbm-idf-20671754

🎯 FIN DU RUN – aucune donnée écrasée.
➡ Tu peux relancer 100x, il reprendra automatiquement où tu t'es arrêté.


## Version Async (lent pas contre)

In [ ]:
RAW_PATH    = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/sampled_data_full_fields_clean.jsonl"
LABELED_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/LLM_labels.jsonl"
REVIEW_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/to_review.jsonl"

N_TO_LABEL = 10          # nbr offres à labelliser
MAX_CONCURRENT = 5          # threads parallèles (limite API)
MAX_CHARS = 10000         # limite tokens, strat utile

# ======== CLEAN TEXT ========
def clean_and_truncate(x):
    if isinstance(x, str):
        x = (x.replace("<p>", " ").replace("</p>", " ")
               .replace("<br/>", " ").replace("\n", " ").strip())
        return x[:MAX_CHARS]
    return ""

# ======== PROMPTS ========
SYSTEM_PROMPT = """
You are an expert HR analyst specialized in job offer analysis.

Your ONLY task is to output STRICT JSON with these EXACT keys:
DIVERSITY, REMUNERATION_BENEFITS, PROFESSIONAL_OPPORTUNITIES,
CULTURE_VALUES, LEADERSHIP, WORK_LIFE_BALANCE, CONFIDENCE.

You MUST NOT write any explanation, comments, or text outside the JSON.

Here is how to assign each label (ONLY ONE 1 or 0 each):

"DIVERSITY":
Repérez si l'offre mentionne l'une des formes de diversité :
- Handicap (accessibilité, adaptations raisonnables, etc.)
- Inclusion & équité (égalité des chances, non-discrimination, etc.)
- Genre (égalité des genres, LGBTQ+, etc.)
- Âge (diversité générationnelle)
Répondez 1 si au moins un de ces points est présent, sinon 0. AUCUN autre texte.

"REMUNERATION_BENEFITS":
Cherchez :
- Salaire, primes, bonus
- Mutuelle, avantages sociaux, retraite
Répondez 1 si au moins un de ces points est présent, sinon 0.

"PROFESSIONAL_OPPORTUNITIES":
Cherchez :
- Formation, mentorat, développement des compétences
- Évolution de carrière, nouveaux projets, mobilité
Répondez 1 si au moins un de ces points est présent, sinon 0.

"CULTURE_VALUES":
Repérez :
- Culture entreprise, valeurs, collaboration, mission, esprit d’équipe
Répondez 1 si au moins un de ces points est présent, sinon 0.

"LEADERSHIP":
Cherchez :
- Mentions de dirigeants, vision stratégique, style de management
Répondez 1 si au moins un de ces points est présent, sinon 0.

"WORK_LIFE_BALANCE":
Repérez :
- Télétravail, horaires flexibles
- Bien-être / équilibre vie pro-vie perso / congés parentaux
Répondez 1 si au moins un de ces points est présent, sinon 0.

You MUST RETURN ONLY valid JSON like:
{
  "DIVERSITY": 0 or 1,
  "REMUNERATION_BENEFITS": 0 or 1,
  "PROFESSIONAL_OPPORTUNITIES": 0 or 1,
  "CULTURE_VALUES": 0 or 1,
  "LEADERSHIP": 0 or 1,
  "WORK_LIFE_BALANCE": 0 or 1,
  "CONFIDENCE": integer 0-100
}
If unsure → still output JSON with 0 everywhere.
ANY OTHER OUTPUT IS WRONG.
"""

def build_user_prompt(row):
    return f"""
Title: {row.get('job_title')}
Company: {row.get('entreprise_nom')}
Location: {row.get('location_label')}
Contract: {row.get('contractType')}
Salary: {row.get('salary_min')} - {row.get('salary_max')} ({row.get('salary_period')})

Job Description:
{clean_and_truncate(row.get('description_job'))}

Profile Required:
{clean_and_truncate(row.get('description_profil'))}

Full Description:
{clean_and_truncate(row.get('description_full'))}

Return ONLY JSON with 7 keys.
"""

# ======== JSON EXTRACTION ROBUSTE (IMPORTANT) ========
def extract_json(text):
    """Isoler le JSON même si GPT écrit autour."""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            return None
    return None

# ======== API CALL ========
semaphore = asyncio.Semaphore(MAX_CONCURRENT)

async def call_llm_async(row):
    async with semaphore:
        t0 = time.time()
        try:
            response = await client.chat.completions.create(
                model="gpt-5-nano",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user",   "content": build_user_prompt(row)},
                ],
            )
            content = response.choices[0].message.content
            try:
                j = json.loads(content)
                return row, j, time.time() - t0
            except:
                return row, None, time.time() - t0
        except Exception:
            return row, None, time.time() - t0


# ========= LOAD RAW =========
df_raw = pd.read_json(RAW_PATH, lines=True)

# ⇨ AVOID RELABELING ALREADY DONE
if os.path.exists(LABELED_PATH):
    done_urls = set(pd.read_json(LABELED_PATH, lines=True)["url"].unique())
else:
    done_urls = set()

df_todo = df_raw[~df_raw["url"].isin(done_urls)].head(N_TO_LABEL).copy()

print(f"🚀 A traiter : {len(df_todo)}")

if len(df_todo) == 0:
    print("🎉 Rien à faire !")
    raise SystemExit


# ========= RUN ASYNC =========
async def main_labeling():
    rows = df_todo.to_dict(orient="records")

    # append files → SÉCURITÉ MAX
    f_out = open(LABELED_PATH, "a", encoding="utf-8")
    f_rev = open(REVIEW_PATH, "a", encoding="utf-8")

    tasks = [call_llm_async(row) for row in rows]

    # ❌ ERREUR : tqdm_asyncio.as_completed(tasks)  → pas un aiterator
    # ✔ SOLUTION :
    for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
        row, res, t_api = await fut   # ← await ici !
        print(f"⚙️ {t_api:.2f}s | {row.get('url')}")

        if res is not None and res.get("CONFIDENCE", 0) >= 50:
            row["Label_All"] = res
            json.dump(row, f_out, ensure_ascii=False); f_out.write("\n"); f_out.flush()
        else:
            row["LLM_PROPOSED_LABELS"] = res
            json.dump(row, f_rev, ensure_ascii=False); f_rev.write("\n"); f_rev.flush()

    f_out.close()
    f_rev.close()
    print("\n🎯 FIN DU RUN – 100% SAFE")


# ========= LAUNCH =========
import nest_asyncio
nest_asyncio.apply()
await main_labeling()

NameError: name 'df_auto' is not defined

In [ ]:
import os, json, time, re, nest_asyncio, asyncio
import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI

# ========= CONFIG =========
RAW_PATH    = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/sampled_data_full_fields_clean.jsonl"
LABELED_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/LLM_labels.jsonl"
REVIEW_PATH = "/Users/noeamar/Documents/HEC/Jocas-HEC/Data/to_review.jsonl"

N_TO_LABEL = 20
MAX_CHARS = 10000
N_CONCURRENT = 8                     # 🔥 simultanés (= VRAI async)
CHUNK_SIZE = N_CONCURRENT * 2        # sécurité

nest_asyncio.apply()                 # indispensable en notebook
semaphore = asyncio.Semaphore(N_CONCURRENT)

# ========= PROMPT =========
# ======== PROMPTS ========
SYSTEM_PROMPT = """
You are an expert HR analyst specialized in job offer analysis.

Your ONLY task is to output STRICT JSON with these EXACT keys:
DIVERSITY, REMUNERATION_BENEFITS, PROFESSIONAL_OPPORTUNITIES,
CULTURE_VALUES, LEADERSHIP, WORK_LIFE_BALANCE, CONFIDENCE.

You MUST NOT write any explanation, comments, or text outside the JSON.

Here is how to assign each label (ONLY ONE 1 or 0 each):

"DIVERSITY":
Repérez si l'offre mentionne l'une des formes de diversité :
- Handicap (accessibilité, adaptations raisonnables, etc.)
- Inclusion & équité (égalité des chances, non-discrimination, etc.)
- Genre (égalité des genres, LGBTQ+, etc.)
- Âge (diversité générationnelle)
Répondez 1 si au moins un de ces points est présent, sinon 0. AUCUN autre texte.

"REMUNERATION_BENEFITS":
Cherchez :
- Salaire, primes, bonus
- Mutuelle, avantages sociaux, retraite
Répondez 1 si au moins un de ces points est présent, sinon 0.

"PROFESSIONAL_OPPORTUNITIES":
Cherchez :
- Formation, mentorat, développement des compétences
- Évolution de carrière, nouveaux projets, mobilité
Répondez 1 si au moins un de ces points est présent, sinon 0.

"CULTURE_VALUES":
Repérez :
- Culture entreprise, valeurs, collaboration, mission, esprit d’équipe
Répondez 1 si au moins un de ces points est présent, sinon 0.

"LEADERSHIP":
Cherchez :
- Mentions de dirigeants, vision stratégique, style de management
Répondez 1 si au moins un de ces points est présent, sinon 0.

"WORK_LIFE_BALANCE":
Repérez :
- Télétravail, horaires flexibles
- Bien-être / équilibre vie pro-vie perso / congés parentaux
Répondez 1 si au moins un de ces points est présent, sinon 0.

You MUST RETURN ONLY valid JSON like:
{
  "DIVERSITY": 0 or 1,
  "REMUNERATION_BENEFITS": 0 or 1,
  "PROFESSIONAL_OPPORTUNITIES": 0 or 1,
  "CULTURE_VALUES": 0 or 1,
  "LEADERSHIP": 0 or 1,
  "WORK_LIFE_BALANCE": 0 or 1,
  "CONFIDENCE": integer 0-100
}
If unsure → still output JSON with 0 everywhere.
ANY OTHER OUTPUT IS WRONG.
"""

def build_user_prompt(row):
    return f"""
Title: {row.get('job_title')}
Company: {row.get('entreprise_nom')}
Location: {row.get('location_label')}
Contract: {row.get('contractType')}
Salary: {row.get('salary_min')} - {row.get('salary_max')} ({row.get('salary_period')})

Job Description:
{clean_and_truncate(row.get('description_job'))}

Profile Required:
{clean_and_truncate(row.get('description_profil'))}

Full Description:
{clean_and_truncate(row.get('description_full'))}

Return ONLY JSON with 7 keys.
"""

# ========= JSON PARSING =========
def extract_json(text):
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except:
            pass
    return None

# ========= API CALL ASYNC =========
async def call_llm(row):
    async with semaphore:
        t0 = time.time()
        try:
            response = await client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": build_user_prompt(row)},
                ],
            )
            js = extract_json(response.choices[0].message.content)
            return row, js, time.time()-t0

        except Exception as e:
            return row, None, time.time()-t0

# ========= MAIN ASYNC =========
async def label_async():
    df_raw = pd.read_json(RAW_PATH, lines=True)
    df_todo = df_raw.head(N_TO_LABEL).to_dict(orient="records")

    # split in CHUNKS to avoid 2000 tasks d'un coup
    for i in range(0, len(df_todo), CHUNK_SIZE):
        batch = df_todo[i:i+CHUNK_SIZE]

        tasks = [call_llm(row) for row in batch]
        results = await asyncio.gather(*tasks)   # ✔️ vrai async

        # WRITE **INCREMENTALLY** → plus JAMAIS écrasé !
        autos, reviews = [], []
        for row, js, _ in results:
            if js is None or js.get("CONFIDENCE", 0) < 50:
                row["LLM_PROPOSED_LABELS"] = js
                reviews.append(row)
            else:
                row["Label_All"] = js
                autos.append(row)

        if autos:
            pd.DataFrame(autos).to_json(LABELED_PATH, orient="records", lines=True, force_ascii=False, mode='a')
        if reviews:
            pd.DataFrame(reviews).to_json(REVIEW_PATH, orient="records", lines=True, force_ascii=False, mode='a')

        print(f"Chunk {i//CHUNK_SIZE+1} terminé — autos:{len(autos)} | reviews:{len(reviews)}")

    print("🏁 FIN de l'async labeling !")


# ========= RUN (NOTEBOOK-SAFE) =========
await label_async()      # ✔️ PAS asyncio.run → fonctionne en notebook

/Users/noeamar/Documents/HEC/.jocas_hec/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CancelledError: 